In [6]:
import requests
import yaml

langs = yaml.safe_load(requests.get("https://raw.githubusercontent.com/github/linguist/master/lib/linguist/languages.yml").content)

In [7]:
import itertools as it

def features():
  yield "alias", "ipython", "Python"
  
  for name, lang in langs.items():
    yield "alias", name, name
    yield "alias", name.lower(), name
    
    for x in lang.get("aliases", []):
      yield "alias", x, name
    
    for x in lang.get("extensions", []):
      yield "ext", x, name
    
    if "codemirror_mime_type" in lang:
      yield "mime", lang["codemirror_mime_type"], name
      yield "editor.codemirror", name, lang["codemirror_mime_type"]
    elif "codemirror_mode" in lang:
      yield "editor.codemirror", name, lang["codemirror_mode"]
    
    if "ace_mode" in lang:
      yield "editor.monaco", name, lang["ace_mode"]


by = {k: 
 {kk: list(map(lambda x: x[2], vv))[0] if k.startswith("editor.") else list(map(lambda x: x[2], vv))
  for kk, vv in it.groupby(v, key=lambda x: x[1])
 }
 for k, v in it.groupby(sorted(features()), key=lambda x: x[0])
}

by["editor"] = {}

for k in list(by.keys()):
  if k.startswith("editor."):
    by["editor"][k[7:]] = by[k]
    del by[k]

by["info"] = {}

for name, lang in langs.items():
    by["info"][name] = info = {"name": name}
    
    if "extensions" in lang:
        info["file_extension"] = lang["extensions"][0]
    
    if "codemirror_mime_type" in lang:
        info["mimetype"] = lang["codemirror_mime_type"]


In [8]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def assign(line, cell):
    """Assign the text contents of the cell to the variable named."""
    get_ipython().user_ns[line.strip()] = cell

In [9]:
%%assign prelude
// Generated by Languages.ipynb

export interface LanguageInfoMetadata {
    name?:            string;
    file_extension?:  string;
    mimetype?:        string;
}

export interface MapToMetadata {
    [key: string]: LanguageInfoMetadata;
}

export interface MapToString {
    [key: string]: string;
}

export interface MapToStrings {
    [key: string]: string[];
}

export interface LanguageData {
    alias:  MapToStrings;
    ext:    MapToStrings;
    mime:   MapToStrings;
    info:   MapToMetadata;
    editor: { [key: string]: MapToString };
}

export const languageData: LanguageData = `DATA`;

In [10]:
import json
import pathlib as fs

fs.Path("languages.ts").write_text(
    prelude.replace("`DATA`",
        json.dumps(by, sort_keys=True, indent=4)
    )
)

261881